In [6]:
import pandas as pd

YEARS = [2018, 2019]

data = pd.DataFrame()

#ingest
for year in YEARS:
    data = data.append(pd.read_csv(f"data/play_by_play_cleaned_{year}.csv.gz", compression='gzip', low_memory=False).reset_index(drop=True), sort=True)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 300)
data.reset_index(drop=True, inplace=True)

In [7]:
# filter data for game script
DOWN_UPPER = 2
NEUTRAL = True
if NEUTRAL:
    neutral = data.loc[ (data.play_type!="no_play") & (data.qtr<5) &
                           (data.down<=DOWN_UPPER) & (data.half_seconds_remaining>120) &
                                     (data.wp>=.2) & (data.wp<=.8)].reset_index(drop=True)
else:
    neutral = data.loc[ (data.play_type!="no_play") & (data.qtr<5)].reset_index(drop=True)

In [10]:

def epa_success(epa):
    return 1 if epa>0 else 0
sea = neutral.loc[(neutral.posteam=="SF")]
qtrData = sea.groupby(['posteam', 'season', 'qtr', 'play_type']).agg(
                play_count = ('play_id', 'count'),
                pass_rate = ('pass', 'mean'),
                total_epa = ('epa', 'sum'),
                avg_epa = ('epa', 'mean'),
                epa_succ_rate = ('epa', lambda x: (x.map(epa_success).mean()))
            ).round(2).reset_index()
qtrData

,posteam,season,qtr,play_type,play_count,pass_rate,total_epa,avg_epa,epa_succ_rate
0,SF,2018,1,pass,86,1,9.46,0.11,0.59
1,SF,2018,1,run,78,0,-14.38,-0.18,0.36
2,SF,2018,2,pass,55,1,22.10,0.40,0.55
3,SF,2018,2,run,58,0,-18.94,-0.33,0.34
4,SF,2018,3,pass,49,1,-5.68,-0.12,0.51
5,SF,2018,3,run,43,0,-4.69,-0.11,0.35
6,SF,2018,4,pass,34,1,-15.09,-0.44,0.50
7,SF,2018,4,run,24,0,-2.82,-0.12,0.38
8,SF,2019,1,pass,99,1,9.32,0.09,0.49
9,SF,2019,1,run,100,0,-4.41,-0.04,0.35
